# Objectifs

* ~~capture multiples photos sur une durée précise~~
* ~~se séparer de l'écriture (=enregistrement) en local de ces photos (pas de photo.jpeg en local)~~
* ~~parvenir à écrire de multiples frames ou les regrouper dans un batch à passer au cnn (sans écrire le batch en local)~~
* ~~avoir un simple script module_1 qui prenne en input la photo, et la restitue en output~~
* **ignorer deepface: fouiller les outils openCV pour détecter la présence d'un visage** -> nice-to-have
* ~~supprimer l'enregistrement vidéo et simplifier la cellule openCV~~
* ~~collectiviser les réponses CNN, extraire la plus fréquente, envoyer au LLM cette réponse AVEC une limite dans le temps de nb de requêtes~~
* réception probas pour émotion (+0 émotion détectée): pondre formule pour isoler émotion principale et envoyer prompt en conséquence, **ou**
    formule pour associer plusieurs émotions et personnaliser chaque prompt pour chaque cas de figure (=16 prompts +gestion incertitude)
* **FORMAT**: output cnn = 4 probas (score de 0 à 3), notre formule doit pouvoir interpréter la plus forte
* Prévoir la gestion d'exception (="incertitude"), trouver le moyen de traiter le scoring inverse d'engagement (="disengagement")
* concevoir le script final d'application + requirements.txt

In [2]:
import numpy
import cv2
from PIL import Image
import time
from collections import deque, Counter

In [1]:
import testcnn
import llm

In [3]:
def showfps(frame, prev_frame_time):
    #parameters
    font = cv2.FONT_HERSHEY_SIMPLEX
    #calculate fps
    new_frame_time = time.time()
    fps = 1/(new_frame_time - prev_frame_time)
    fps = str(round(fps))
    cv2.putText(frame, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
    return new_frame_time

In [4]:
def get_response_from_cnn(frame):
    pilimage = Image.fromarray(frame).convert("RGB")
    cnnresponse = testcnn.testrestitution(pilimage)
    return cnnresponse

In [5]:
def evaluate_response(history):
    return Counter(history).most_common(1)[0][0]

In [8]:
def perform_action(action):
    print(f"[LLM] Message envoyé pour l’émotion détectée : {action}")

    if action == "boredom":
        print("LLM: Tu t'ennuies, une courte pause peut t’aider.")
    elif action == "engagement":
        print("LLM: T'es à fond, tu déchires tout!")
    elif action == "disengagement":
        print("LLM: Hey ! Reviens avec nous, tu peux le faire.")
    elif action == "confusion":
        print("LLM: Je suis là pour t'aider, prenons les choses une étape à la fois.")
    elif action == "frustration":
        print("LLM: Prends une pause pour te reposer.")
    elif action == "incertitude":
        print("LLM: Je suis avec toi, on va y arriver.")

In [ ]:
cap = cv2.VideoCapture(0)
prev_frame_time = 0
new_frame_time = 0
deq_len = 100
history = deque([], maxlen=deq_len)
last_action_time = time.time()

while( cap.isOpened() ):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.flip(frame,1)
        prev_frame_time = showfps(frame, prev_frame_time)

        cnnresponse = get_response_from_cnn(frame)
        #print(cnnresponse)
        ###stop calcul
        history.append(cnnresponse)

        if len(history) == deq_len:
            action = evaluate_response(history)

            if time.time() - last_action_time >= 10:
                last_action_time = time.time()
                print(action)
                message = llm.get_recommendation(action)
                print(message)

        cv2.imshow('frame' , frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

engagement
"Continue comme ça, tu es sur la bonne voie !"


In [24]:
# Initialize webcam (0 = default camera)
cam = cv2.VideoCapture(0)

# Capture one frame
ret, frame = cam.read()

if ret:
    cv2.imwrite("captured_image.jpeg", frame)  
else:
    print("Failed to capture image.")

cam.release()

In [ ]:
%load_ext autoreload
%autoreload 2

2